In [1]:
## fix auto reloading
import pandas
import os
import sys
from dotenv import load_dotenv
import json
import pandas as pd
load_dotenv()
from video_selection.google_trends import get_trends
from video_selection.search_by_keyword import write_keyword_and_id
from video_selection.filter import filter_sample_videos
from metadata_extraction.extract_metadata import metadata_extraction
from metadata_extraction.extract_metadata import youtube_authenticate
from datetime import datetime


In [3]:
# pipeline setup
keyword_path = './input/input_keyword_list.csv' # Modified for different disease, Input the keyword list
keyword_ref = 'sleep apnea causes' # use the first keyword for comparison in each iteration. 
num_of_result = 100

# Video Selection

In [4]:
# test for google trends
keyword_list = get_trends(keyword_path, keyword_ref, num_of_result)
print(keyword_list[:10])

['Pain', 'anxiety', 'Sleeping', 'night', 'sore throat', 'coughing', 'Stress', 'weight gain', 'dry mouth', 'acid reflux']


/Users/ZHANGRY/Codes/youtube-analytics-pipeline/video_selection/google_trends.py:109: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_popularity = normalised.mean().sort_values(ascending = False).head(num_result)


In [ ]:
# test search for keyword
df_keyword = write_keyword_and_id(keyword_list, 5)
df_keyword.head(5)

In [23]:
df_keyword

,keyword,id,rank,toKeep
304,excessive yawning,-MjaUhSOf94,5,False
367,difficulty falling asleep,PmM1IH-LaVQ,3,False
394,Obstructive Syndrome,krxcVBgerHM,5,False
480,pain in front center of brain,FSPzEzNBtMg,1,False
481,pain in front center of brain,HvVa7vUSRIo,2,False
482,pain in front center of brain,1JrdO-1Wf8g,3,False
483,pain in front center of brain,TYhgbE_84MA,4,False
484,pain in front center of brain,vPje60GgHXc,5,False


In [ ]:
# test filter and sample
credential_path = "oauth_credential.json"
# just for test
df_keyword = pd.read_csv('./temp/videoID_list.csv')
sampled_videos = filter_sample_videos(df_keyword, credential_path)
sampled_videos.head(10)

In [17]:
# display the video_dictionary
df_video = pd.read_csv('temp/sampled_filtered_video_list.csv')
df_video.head()

,keyword,id,rank
0,ACROMEGALY,KLoaYPXlWHM,22
1,ACROMEGALY,OFl_bKVYsPo,9
2,ACROMEGALY,DAWYyBQJ8Jk,7
3,ACROMEGALY,AI-edvZx2GU,23
4,Anxious mood,YufQY6HqCW0,4


In [25]:
df_video[df_video['id']== "KLoaYPXlWHM"].index[0]

0

# Extract Metadata

In [11]:
# test setup
testID = "KLoaYPXlWHM"
test_keyword = "ACROMEGALY"
OAUTH_CREDENTIAL_PATH = "credentials/oauth_credential_test.json"
SERVICE_ACCOUNT_PATH = "credentials/service_account.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + SERVICE_ACCOUNT_PATH
youtube = youtube_authenticate(OAUTH_CREDENTIAL_PATH)

In [10]:
# extract raw metadata
youtube = youtube_authenticate(OAUTH_CREDENTIAL_PATH)
raw_metadata = metadata_extraction(youtube, testID) # in json file
pd.DataFrame(raw_metadata, index=[0])

extracting the id:  KLoaYPXlWHM
Retrieving accTag for:  https://www.youtube.com/watch?v=KLoaYPXlWHM
error in extracting accTag


,id,titleLength,hasTags,num_of_tags,hasDescription,hasThumbnail,channel_subscribers,accreditationTag,duration,description,publish_days
0,KLoaYPXlWHM,11,1,17,1,1,33400,0,259.0,Part II of The Voices of Acromegaly video seri...,2998


In [35]:
# video feature extraction
# one video takes roughly 4 minutes
os.environ["SERVICE_ACCOUNT_PATH"] = "/credentials/service_account.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + os.environ["SERVICE_ACCOUNT_PATH"]
from video_analysis.video_analysis import analyze_by_path
def id_2_index(videoID):
    return df_video[df_video['id']== videoID].index[0]

VIDEO_BUCKET_PATH = "gs://youtube-video-bucket/"
def index_2_gcp_path(index):
    return VIDEO_BUCKET_PATH + str(index) + ".mp4"

test_video_path = index_2_gcp_path(id_2_index(testID))
video_feature = analyze_by_path(test_video_path)
pd.DataFrame(video_feature, index=[0])


Processing video for shot change annotations:

Finished processing.

Processing video for object annotations.

Finished processing.


Processing video for text detection.


,num_of_shots,num_of_objects,text_confidence
0,9,145,0.97195


In [ ]:
# nlp feature
# transcription features

# cosine similarty calculation


In [ ]:
# cosine similarty calculation


In [6]:
# final concatenation


'{"installed":{"client_id":"45152107222-al8csdiq5n470pknclhuv4a57gjr6h6r.apps.googleusercontent.com","project_id":"geometric-rock-358702","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"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","redirect_uris":["http://localhost"]}}'

In [29]:
print(os.environ)

environ({'COMMAND_MODE': 'unix2003', 'CONDA_DEFAULT_ENV': 'ytbenv', 'CONDA_EXE': '/opt/anaconda3/bin/conda', 'CONDA_PREFIX': '/opt/anaconda3/envs/ytbenv', 'CONDA_PROMPT_MODIFIER': '(ytbenv) ', 'CONDA_PYTHON_EXE': '/opt/anaconda3/bin/python', 'CONDA_SHLVL': '2', 'HOME': '/Users/ZHANGRY', 'HOMEBREW_CELLAR': '/opt/homebrew/Cellar', 'HOMEBREW_PREFIX': '/opt/homebrew', 'HOMEBREW_REPOSITORY': '/opt/homebrew', 'INFOPATH': '/opt/homebrew/share/info:', 'JAVA_HOME': '/Users/ZHANGRY/Library/Java/JavaVirtualMachines/jdk-18.0.2.1.jdk/Contents/Home', 'LESS': '-R', 'LOGNAME': 'ZHANGRY', 'LSCOLORS': 'Gxfxcxdxbxegedabagacad', 'MANPATH': '/Users/ZHANGRY/.nvm/versions/node/v18.8.0/share/man:/opt/homebrew/share/man::', 'MallocNanoZone': '0', 'NVM_BIN': '/Users/ZHANGRY/.nvm/versions/node/v18.8.0/bin', 'NVM_CD_FLAGS': '-q', 'NVM_DIR': '/Users/ZHANGRY/.nvm', 'NVM_INC': '/Users/ZHANGRY/.nvm/versions/node/v18.8.0/include/node', 'OLDPWD': '/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'P9K_SSH': '0', 'PAGER'

In [31]:
os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

'/Users/ZHANGRY/Codes/youtube-analytics-pipeline/credential_and_key/geometric-rock-358702-c152672f14dc.json'

In [32]:
os.getenv("AUDIO_BUCKET_NAME")